# Full Pipeline 테스트

전체 분석 파이프라인 (Supervisor Pattern) 테스트 노트북

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

In [ ]:
SAMPLE_STORY = """
어두운 숲 속에서 서진은 검을 쥐고 있었다. 그는 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
"우리가 이곳에서 그를 만날 수 있을까?" 서진의 옆에 서 있던 하나가 물었다.
하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다.

갑자기 나무 사이에서 그림자가 나타났다. 그것은 이민호였다. 한때는 서진의 가장 친한 친구였지만,
지금은 그들의 마을을 배신한 적이 되어버렸다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. "아직도 그 낡은 검을 들고 다니는군."

서진은 용감하게 앞으로 나섰다. "왜 마을을 배신했어, 민호?"

이민호는 미소지었다. 그의 눈에는 슬픔과 분노가 섞여 있었다.
"넌 모르지. 마을 장로들이 내 가족에게 무슨 짓을 했는지."

하나가 두 사람 사이로 걸어나왔다. "싸우기 전에 대화를 해봐요. 아직 늦지 않았어요."

박서연이라는 젊은 여성이 숲에서 뛰어나왔다. 그녀는 마을의 정탐꾼으로, 빠르고 민첩했다.
"경비병들이 오고 있어요! 빨리 결정해야 해요!"
"""

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

In [ ]:
from app.agents.graph import run_analysis_pipeline

async def test_full_pipeline():
    print("🚀 Full Pipeline 테스트 시작...")
    print("(시간이 오래 걸릴 수 있습니다)\n")
    
    result = await run_analysis_pipeline(
        content=SAMPLE_STORY,
        project_id="test-project-001",
        document_id="test-document-001",
        job_id="test-job-001",
        callback_url="http://localhost:8080/api/callback"
    )
    return result

result = run_async(test_full_pipeline())
print(json.dumps(result, ensure_ascii=False, indent=2))

In [ ]:
# 결과 분석
if result:
    print("\n=== 파이프라인 결과 요약 ===")
    print(f"캐릭터: {len(result.get('extracted_characters', []))}명")
    print(f"이벤트: {len(result.get('extracted_events', []))}개")
    print(f"관계: {len(result.get('relationship_graph', {}).get('relationships', []))}개")
    print(f"일관성 점수: {result.get('consistency_report', {}).get('overall_score', 'N/A')}")
    print(f"검증 결과: {result.get('validation_result', {}).get('action', 'N/A')}")
    print(f"에러: {result.get('errors', [])}")